In [2]:
%load_ext autoreload
%autoreload 2

In [17]:
import numpy as np
import pandas as pd

import torch

from transformers import AutoTokenizer, GPT2LMHeadModel, set_seed

from src.config import Config

CONFIG:Config = Config()
set_seed(CONFIG.seed)

In [43]:
df = pd.read_pickle(CONFIG.dataset.train_data_raw)
df.head()

,whoTarget,intentYN,sexYN,offensiveYN,speakerMinorityYN,post,targetMinority,targetCategory,targetStereotype,dataSource
0,0.0,1.0,0.0,1.0,0.0,RT @_LexC__: I'm convinced that some of y'all ...,,,,t/davidson
1,0.0,1.0,0.0,1.0,0.0,RT @_LexC__: I'm convinced that some of y'all ...,,,,t/davidson
2,0.0,0.0,0.0,1.0,0.0,RT @_LexC__: I'm convinced that some of y'all ...,,,,t/davidson
3,1.0,1.0,0.0,1.0,0.0,RT @iBeZo: Stupid fucking nigger LeBron. You f...,black folks,race,all stupid,t/davidson
4,1.0,1.0,0.0,1.0,0.0,RT @iBeZo: Stupid fucking nigger LeBron. You f...,black folks,race,are not people but apes.,t/davidson


In [55]:
class_label_encoder = {
    0: {0: "[grpN]", 1: "[grpY]"},
    1: {0: "[intN]", 1: "[intY]"},
    2: {0: "[sexN]", 1: "[sexY]"},
    3: {0: "[offN]", 1: "[offY]"},
    4: {0: "[ingrpN]", 1: "[ingrpY]"},
}

In [68]:
t = df.to_numpy()[0][:5]
t_enc = [class_label_encoder[idx][a] for idx,a in enumerate(t)]
"[SEP]".join(t_enc[:4]) + "[SEP]" + "ciao come stai" + "[SEP]" + "bene tu" + "[SEP]" + t_enc[-1]

'[grpN][SEP][intY][SEP][sexN][SEP][offY][SEP]ciao come stai[SEP]bene tu[SEP][ingrpN]'

In [11]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [40]:
tokenizer(text="ciao sono Matteo", )

1024

In [60]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
# encoder = AutoTokenizer.from_pretrained('gpt2')
SPECIAL_TOKENS = {
    'pad_token': '[PAD]',
    'sep_token': '[SEP]',
    "additional_special_tokens": ["[offY]", "[offN]", "[sexY]", "[sexN]", "[intY]", 
                                   "[intN]", "[grpY]", "[grpN]", "[ingrpN]", "[ingrpY]"]
}
tokenizer.add_special_tokens(SPECIAL_TOKENS)
print(tokenizer.sep_token, tokenizer.sep_token_id)
print(tokenizer.pad_token, tokenizer.pad_token_id)
print('All special tokens:', tokenizer.all_special_tokens)
print('All special ids:', tokenizer.all_special_ids)

[SEP] 50258
[PAD] 50257
All special tokens: ['<|endoftext|>', '[SEP]', '[PAD]', '[offY]', '[offN]', '[sexY]', '[sexN]', '[intY]', '[intN]', '[grpY]', '[grpN]', '[ingrpN]', '[ingrpY]']
All special ids: [50256, 50258, 50257, 50259, 50260, 50261, 50262, 50263, 50264, 50265, 50266, 50267, 50268]


In [32]:
a = tokenizer("ciao sono bello", "anche tu", max_length=15, padding="max_length", add_special_tokens=True)
print(a)
print(tokenizer.decode(a["input_ids"], skip_special_tokens=False))
model.config.eos_token_id

{'input_ids': [66, 13481, 3367, 78, 8966, 78, 6362, 12777, 50257, 50257, 50257, 50257, 50257, 50257, 50257], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]}
ciao sono belloanche tu[PAD][PAD][PAD][PAD][PAD][PAD][PAD]


In [36]:
model.config.pad_token_id

In [21]:
from datasets import Dataset

df_train = pd.read_csv("data/old_agg/SBIC.v2.agg.trn.csv")
df_val = pd.read_csv("data/old_agg/SBIC.v2.agg.dev.csv")

dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
train_tokenized = dataset_train.map(tokenize_function, batched=True)
val_tokenized = df_val.map(tokenize_function, batched=True)


train_tokenized = train_tokenized.shuffle(seed=42).select(range(1000))
val_tokenized = val_tokenized.shuffle(seed=42).select(range(1000))

NameError: name 'tokenize_function' is not defined

In [ ]:





accelerator = Accelerator(mixed_precision=config.mixed_precision, cpu=config.cpu)
(
    model,
    optimizer,
    train_dataloader,
    val_dataloader,
    lr_scheduler,
) = accelerator.prepare(
    model, optimizer, train_dataloader, val_dataloader, lr_scheduler
)
if isinstance(loss_fn, nn.Module):
    loss_fn = accelerator.prepare(loss_fn)

# Run training and track with wandb
steps_per_epoch = len(train_dataloader)
total_steps = steps_per_epoch * config.num_epochs

checkpoint_counter = 0
step = 0
avg_loss = AvgValue()
avg_inner_losses = defaultdict(AvgValue)
model.train()

forward_signature = set(inspect.signature(model.forward).parameters)
progress_bar = tqdm(range(total_steps))
for epoch in range(config.num_epochs):
    for data in train_dataloader:
        inputs = {
            argument: value
            for argument, value in data.items()
            if argument in forward_signature
        }

        loss, inner_losses = train_batch(
            inputs=inputs,
            data=data,
            step=step,
            model=model,
            loss_fn=loss_fn,
            optimizer=optimizer,
            lr_scheduler=lr_scheduler,
            teacher_force_scheduler=teacher_force_scheduler,
            accelerator=accelerator,
            config=config,
        )
        progress_bar.update(1)

        # Compute statistics
        n_samples = len(next(iter(data.values())))
        step += 1
        avg_loss.update(loss, n_samples)
        for loss_name, loss_value in inner_losses.items():
            avg_inner_losses[f"avg_{loss_name}"].update(loss_value, n_samples)

        wandb.log(
            {
                "train_loss": loss,
                **inner_losses,
                "lr": lr_scheduler.get_last_lr()[0],
            },
            step=step,
        )

        # Evaluate the model and save checkpoints
        if (step % config.log_interval == 0) or (step == total_steps):
            # Evaluate the model
            val_loss, val_inner_losses, val_metrics = train_evaluation(
                model,
                val_dataloader,
                loss_fn,
                # metrics=metrics,
            )
            model.train()

            train_log(
                avg_loss,
                avg_inner_losses,
                val_loss,
                val_inner_losses,
                val_metrics,
                lr=lr_scheduler.get_last_lr()[0],
                step=step,
            )
            avg_loss = AvgValue()
            avg_inner_losses = defaultdict(AvgValue)

        if step % config.checkpoint_interval == 0:
            # Saving checkpoint
            save_model_checkpoint(
                model,
                optimizer,
                lr_scheduler,
                epoch,
                step,
                checkpoint_counter,
                config,
            )
            wandb.log(
                {
                    "checkpoint_counter": checkpoint_counter,
                },
                step=step,
            )
            checkpoint_counter += 1

    gc.collect()
    torch.cuda.empty_cache()